In [ ]:
! pip install -U accelerate
! pip install -U transformers

import os
os._exit(00)

In [1]:
# Connect to Google Drive and upload a folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
# import torch.optim as optim
import re # Regular expression
from tqdm import tqdm

from torch.utils.data import Dataset
from typing import List, Dict, Union
from transformers import pipeline
from transformers import Trainer, TrainingArguments, AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification#, BertForTokenClassification

import gc

In [44]:
# MODEL_NAME = 'bert-base-cased'
MODEL_NAME = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
tokenizer.add_special_tokens({'pad_token': '-100 '})

model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=7)

Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Downloading Data

In [3]:
# !git clone https://github.com/s-nlp/semantic-role-labelling.git

In [4]:
path_train = 'https://raw.githubusercontent.com/profii/srl_transformers/main/dataset/train.tsv'


In [5]:
df = pd.read_csv(path_train, sep='\t', header= None, names=['data', 'label'],
                 quoting=3, skip_blank_lines=False).fillna('_nan')

In [6]:
df.head(22)

,data,label
0,also,O
1,",",O
2,i,O
3,have,O
4,recently,O
5,discovered,O
6,advil,B-Object
7,liquigels,O
8,work,O
9,much,O


In [9]:
df.shape

(63408, 2)

In [10]:
df.data[592]

'"'

## Preprocessing

    - Separating data into sentences with empty lines (NaN).
    - Clean punctuation into single dot.
    - Reconstruct labels into [0, 1, 2, 3, 4, 5, 6]

In [21]:
# Separating data into sentences with empty lines (NaN)

def separate_text(df):
    sents = []
    tags = []
    sentence = []
    label = []

    for word, tag in df.values:
        if word == '_nan':
            sents.append(sentence)
            tags.append(label)
            sentence = []
            label = []
        else:
            word = re.sub(r"[\"\—\#\$\%\&\'\(\)\*\+\,\–\-\/\:\;\<\=\>\?\@\[\\\]\^\?\!\_\`\{\|\}\~\«\»ѣ\№]", ".", word)
            word = re.sub(r"[.]+", ".", word)
            sentence.append(word)
            label.append(tag)
            # label.append(labels_to_ids[tag])

    return sents, tags

In [22]:
labels_to_ids = {k: v for v, k in enumerate(['O', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'])}
ids_to_labels = {v: k for v, k in enumerate(['O', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'])}

# Appling cleaning to df
sents, tags = separate_text(df)

In [23]:
labels_to_ids

{'O': 0,
 'B-Object': 1,
 'I-Object': 2,
 'B-Aspect': 3,
 'I-Aspect': 4,
 'B-Predicate': 5,
 'I-Predicate': 6}

In [ ]:
# sents[0], tags[0]

In [47]:
input_text = "Once upon a time"
input_ids = tokenizer(input_text, return_tensors='pt', padding=True)['input_ids']

In [35]:
def align_label(texts, labels, max_length=150, label_all_tokens=True):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=max_length, is_split_into_words=True)

    l = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)

            elif word_idx != previous_word_idx:
                label_ids.append(labels_to_ids[label[word_idx]])

            else:
                label_ids.append(labels_to_ids[label[word_idx]] if label_all_tokens else -100)

            previous_word_idx = word_idx
        l.append(label_ids)
    tokenized_inputs["labels"] = l

    return tokenized_inputs

In [34]:
print(tags)

[['O', 'O', 'O', 'O', 'O', 'O', 'B-Object', 'O', 'O', 'O', 'B-Predicate', 'O', 'B-Predicate', 'O', 'O', 'B-Aspect', 'O', 'O', 'B-Object', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-Object', 'O', 'B-Predicate', 'O', 'B-Object', 'O', 'B-Aspect', 'O', 'O', 'O', 'B-Object', 'O', 'B-Predicate', 'O', 'B-Aspect', 'I-Aspect', 'O', 'B-Aspect', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Object', 'O', 'B-Predicate', 'I-Predicate', 'O', 'B-Object', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-Object', 'O', 'O', 'B-Predicate', 'O', 'B-Object', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Object', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Object', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-Object', 'O', 'O', 'B-Predicate', 'O', 'O', 'O', 'B-Object', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Object', 

In [36]:
align_label(sents[:1], tags[:1])

AssertionError: ignored

#Loading all neccessary, Making input and Learning

In [ ]:
!pip install seqeval
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f53231bf87490c9c5daf4890e638789fdcba037d1dcb52327752f30ba30c83a7
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [20]:
from datasets import load_metric

metric = load_metric("seqeval")

ModuleNotFoundError: ignored

In [ ]:
def compute_metrics(eval_preds):
    # print(eval_preds)

    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)
    # tokenizer.convert_ids_to_tokens(
    predictions = [
        [ids_to_labels[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
        [ids_to_labels[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
        }
    # return {"f1": results["overall_f1"]}

In [ ]:
class PairsDataset(Dataset):
    def __init__(self, x):
        self.y = x['labels']
        del x['labels']
        self.x = x

    def __getitem__(self, idx):
        assert idx <= len(self.x['input_ids']), (idx, len(self.x['input_ids']))
        item = {key: val[idx] for key, val in self.x.items()}
        item['labels'] = self.y[idx]

        return item

    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n

In [ ]:
train_dataset = PairsDataset(align_label(sents,tags))
dev_dataset = PairsDataset(align_label(sents_dev,tags_dev))

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
temp_f = train_dataset[0]

z=0
for i, j, k, c in zip(temp_f['attention_mask'],
                    tokenizer.convert_ids_to_tokens(temp_f['input_ids']),
                    temp_f['labels'],
                    temp_f['input_ids']):

    z+=1
    if z == 40:
        break
    if k == -100:
        print(i, j, '\t\t -100 \t\t', c, k)
    else:
        print(i, j, '\t\t', ids_to_labels[k], '\t\t', c, k)

1 [CLS] 		 -100 		 101 -100
1 also 		 O 		 2036 0
1 . 		 O 		 1012 0
1 i 		 O 		 1045 0
1 have 		 O 		 2031 0
1 recently 		 O 		 3728 0
1 discovered 		 O 		 3603 0
1 ad 		 B-Object 		 4748 1
1 ##vil 		 B-Object 		 14762 1
1 li 		 O 		 5622 0
1 ##qui 		 O 		 15549 0
1 ##gel 		 O 		 12439 0
1 ##s 		 O 		 2015 0
1 work 		 O 		 2147 0
1 much 		 O 		 2172 0
1 better 		 B-Predicate 		 2488 5
1 and 		 O 		 1998 0
1 faster 		 B-Predicate 		 5514 5
1 for 		 O 		 2005 0
1 a 		 O 		 1037 0
1 headache 		 B-Aspect 		 14978 3
1 than 		 O 		 2084 0
1 regular 		 O 		 3180 0
1 ib 		 B-Object 		 21307 1
1 ##up 		 B-Object 		 6279 1
1 ##ro 		 B-Object 		 3217 1
1 ##fen 		 B-Object 		 18940 1
1 . 		 O 		 1012 0
1 [SEP] 		 -100 		 102 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100
0 [PAD] 		 -100 		 0 -100


In [ ]:
N_EPOCHS = 2
BATCH_SIZE = 4

args = TrainingArguments(output_dir="logs/model",
                         num_train_epochs=N_EPOCHS,
                         per_device_train_batch_size=BATCH_SIZE,
                         per_device_eval_batch_size=BATCH_SIZE,
                         save_steps=10000000,
                         logging_steps=200,
                        #  load_best_model_at_end = False,
                         evaluation_strategy = 'epoch',
                        #  optim='adamw_torch',
                        #  weight_decay=0.01,
                         )


trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.192100,0.312848,0.690150,0.716505,0.703080,0.894923
2,0.104100,0.376948,0.750366,0.663430,0.704225,0.896067


TrainOutput(global_step=1168, training_loss=0.15609894794960544, metrics={'train_runtime': 185.265, 'train_samples_per_second': 25.196, 'train_steps_per_second': 6.304, 'total_flos': 368383837496400.0, 'train_loss': 0.15609894794960544, 'epoch': 2.0})

# Evaluate

In [ ]:
path_dev = 'https://raw.githubusercontent.com/profii/srl_transformers/main/dataset/dev.tsv'
path_test = 'https://raw.githubusercontent.com/profii/srl_transformers/main/dataset/test_no_answers.tsv'

df_dev = pd.read_csv(path_dev, sep='\t', header= None, names=['data', 'labels'], quoting=3)

df_devo = pd.read_csv(path_dev, sep='\t', header= None, names=['data', 'labels'],
                      quoting=3, skip_blank_lines=False).fillna('_nan')

df_test = pd.read_csv(path_test, sep='\t', header= None, names=['data'], quoting=3)

df_testo = pd.read_csv(path_test, sep='\t', header= None, names=['data'],
                      quoting=3, skip_blank_lines=False).fillna('_nan')

In [13]:
sents_dev, tags_dev = separate_text(df_dev)

NameError: ignored

In [ ]:
sents_dev[0], tags_dev[0]

In [ ]:
#ner for pipeline determine task to perform

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
print("Aloha guys".device())

AttributeError: ignored

In [ ]:
def evaluate(nlp, dfo, df):
    indexes_nan = []
    labels_list = []
    sents = separate_text_end(dfo)

    for sent in tqdm(sents):
        result = nlp(' '.join(sent))
        labels_list.extend([ids_to_labels[int(i['entity'][-1:])] for i in result if '#' not in i['word']])
        indexes_nan.append(len(labels_list))

    print(len(labels_list))
    df['labels'] = [labels_list[i] for i in range(df.shape[0])]

    return df, indexes_nan

In [ ]:
# Separating data into sentences with empty lines (NaN)

def separate_text_end(df):
    sents = []
    sentence = []

    for word in df['data']:
        if word == '_nan':
            sents.append(sentence)
            sentence = []
        else:
            word = re.sub(r"[\"\—\#\$\%\&\'\(\)\*\+\,\–\-\/\:\;\<\=\>\?\@\[\\\]\^\?\!\_\`\{\|\}\~\«\»ѣ\№]", ".", word)
            word = re.sub(r"[.]+", ".", word)
            sentence.append(word)

    return sents

In [ ]:
# model_name = 'dev'
# df, indexes_nan = evaluate(nlp, df_devo, df_dev)

model_name = 'test'
df, indexes_nan = evaluate(nlp, df_testo, df_test)

  0%|          | 0/360 [00:00<?, ?it/s]


RuntimeError: ignored

# Saving

In [ ]:
dir = 'bert'

model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/SLR_project/"+dir)

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/SLR_project/"+dir+"_tok")

('/content/drive/MyDrive/Colab Notebooks/SLR_project/bert_tok/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/SLR_project/bert_tok/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/SLR_project/bert_tok/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/SLR_project/bert_tok/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/SLR_project/bert_tok/tokenizer.json')

In [ ]:
df_devo.drop('labels', axis='columns', inplace=True)

In [ ]:
df_test.head()

,data
0,plus
1,","
2,android
3,is
4,developing


In [ ]:
df_testo.head(25)

,data
0,plus
1,","
2,android
3,is
4,developing
5,a
6,way
7,faster
8,than
9,ios


In [ ]:
df_dev.head()

,data,labels
0,meanwhile,O
1,",",O
2,though,O
3,windows,B-Object
4,8,I-Object


In [ ]:
df_devo.head()

,data
0,meanwhile
1,","
2,though
3,windows
4,8


In [ ]:
df_test.shape, df_testo.shape

((9444, 1), (9804, 1))

In [ ]:
df_dev.shape, df_devo.shape

((8363, 2), (8646, 2))

In [ ]:
print(indexes_nan)

NameError: ignored

In [ ]:
df.head(50)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP_project/results/'+dir+'_'+model_name+'.tsv',
          header=None, index=False, quoting=3, sep='\t', encoding='utf-8')

## Save to file

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_project/results/'+dir+'_'+model_name+'.tsv') as input:
    lines = [line for line in input if line.strip()]

with open('/content/drive/MyDrive/Colab Notebooks/NLP_project/results/'+dir+'_'+model_name+'_post.tsv', 'w') as output:
    i = 0
    for line in lines:
        output.write(line)
        if i+1 in (indexes_nan):
            # print(line)
            output.write("\n")
        i += 1

# Load model

In [ ]:
saved_model_name = 'bert_2ep_4b'
# saved_model_name = saved_name

dir = 'bert/'+saved_model_name

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/SLR_project/"+dir)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/SLR_project/"+dir+"_tok")



HFValidationError: ignored